# Limpieza

In [1]:
import pandas as pd
import re
from sklearn.impute import KNNImputer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

In [3]:
%pwd

'c:\\Users\\ADMIN\\Desktop\\Proyecto Final de Maestria\\Notebooks'

In [2]:
df_Hospedajes = pd.read_excel('../data_out/Hospedajes_RD.xlsx')

In [3]:
df_Hospedajes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5038 entries, 0 to 5037
Data columns (total 23 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   ID                       5038 non-null   int64  
 1   titulo                   5025 non-null   object 
 2   Descripcion_alojamiento  5038 non-null   object 
 3   tipo_hospedaje           5027 non-null   object 
 4   Ubicacion                5038 non-null   object 
 5   precio                   4877 non-null   object 
 6   max_personas             4748 non-null   float64
 7   habitaciones             4722 non-null   float64
 8   camas                    4045 non-null   float64
 9   baños                    4442 non-null   float64
 10  super_anfitrion          5038 non-null   object 
 11  cantidad_evaluaciones    2229 non-null   float64
 12  rating_total             2229 non-null   float64
 13  limpieza                 2534 non-null   float64
 14  veracidad               

Cambiamos nombre de columnas a nombres mas intuitivos

In [4]:
df_Hospedajes = df_Hospedajes.rename(columns={
    'ID': 'ID_ALOJAMIENTO',
    'titulo': 'TITULO_ALOJAMIENTO', 
    'Descripcion_alojamiento':'TIPO_ALOJAMIENTO',
    'tipo_hospedaje': 'ESPACIO_ALOJAMIENTO', 
    'Ubicacion':'UBICACION',
    'precio': 'PRECIO_USD', 
    'max_personas': 'MAX_HUESPEDES', 
    'habitaciones': 'CANTIDAD_HABITACIONES', 
    'camas': 'CANTIDAD_CAMAS', 
    'baños': 'CANTIDAD_BAÑOS', 
    'super_anfitrion':'ES_SUPER_ANFITRION',
    'cantidad_evaluaciones': 'CANTIDAD_EVALUACIONES', 
    'rating_total': 'CALIFICACION_GENERAL', 
    'limpieza': 'CALIFICACION_LIMPIEZA', 
    'veracidad': 'CALIFICACION_VERACIDAD', 
    'llegada': 'CALIFICACION_LLEGADA', 
    'comunicacion': 'CALIFICACION_COMUNICACION', 
    'ubicacion_calif': 'CALIFICACION_UBICACION', 
    'calidad_precio': 'CALIFICACION_CALIDAD_PRECIO',
    'Reservacion_Inmediata':'FLAG_RESERV_INMEDIATA',
    'Llegada_Autonoma':'FLAG_LLEGADA_AUTONOMA',
    'Cancelacion_Gratuita':'FLAG_CANCELAC_GRATUITA',
    'Aceptan_Mascotas':'FLAG_ACEPT_MASCOTAS'
})

In [5]:
df_Hospedajes.columns

Index(['ID_ALOJAMIENTO', 'TITULO_ALOJAMIENTO', 'TIPO_ALOJAMIENTO',
       'ESPACIO_ALOJAMIENTO', 'UBICACION', 'PRECIO_USD', 'MAX_HUESPEDES',
       'CANTIDAD_HABITACIONES', 'CANTIDAD_CAMAS', 'CANTIDAD_BAÑOS',
       'ES_SUPER_ANFITRION', 'CANTIDAD_EVALUACIONES', 'CALIFICACION_GENERAL',
       'CALIFICACION_LIMPIEZA', 'CALIFICACION_VERACIDAD',
       'CALIFICACION_LLEGADA', 'CALIFICACION_COMUNICACION',
       'CALIFICACION_UBICACION', 'CALIFICACION_CALIDAD_PRECIO',
       'FLAG_RESERV_INMEDIATA', 'FLAG_LLEGADA_AUTONOMA',
       'FLAG_CANCELAC_GRATUITA', 'FLAG_ACEPT_MASCOTAS'],
      dtype='object')

Aquellos registros vacios lo identificamos como "No encontrado"

In [6]:
columnas = [
    'TITULO_ALOJAMIENTO', 'ESPACIO_ALOJAMIENTO', 'PRECIO_USD', 'MAX_HUESPEDES', 'CANTIDAD_HABITACIONES', 
    'CANTIDAD_CAMAS', 'CANTIDAD_BAÑOS', 'CANTIDAD_EVALUACIONES', 'CALIFICACION_GENERAL', 'CALIFICACION_LIMPIEZA', 
    'CALIFICACION_VERACIDAD', 'CALIFICACION_LLEGADA', 'CALIFICACION_COMUNICACION', 'CALIFICACION_UBICACION', 'CALIFICACION_CALIDAD_PRECIO'
]

# Imputar valores faltantes
for columna in columnas:
    df_Hospedajes[columna] = df_Hospedajes[columna].fillna('No encontrado')


Tipo de alojamiento

In [7]:
# Definir una función para categorizar el tipo de alojamiento
def categorizar_alojamiento(tipo):
    if 'habitacion' in tipo.lower() or 'habitación' in tipo.lower():
        return 'Habitación'
    else:
        return 'Alojamiento entero'

# Aplicar la función a la columna 'TIPO_ALOJAMIENTO'
df_Hospedajes['ESPACIO_ALOJAMIENTO'] = df_Hospedajes['ESPACIO_ALOJAMIENTO'].apply(categorizar_alojamiento)

Ubicacion

In [14]:
# Definir el diccionario de ubicaciones específicas a ciudades
ubicaciones_a_ciudades = {
    "Santo Domingo": "Santo Domingo",
    "Punta Cana": "Punta Cana",
    "Boca Chica": "Boca Chica",
    "Juan Dolio": "Juan Dolio",
    "Juandolio":"Juan Dolio",
    "Puerto Plata": "Puerto Plata",
    "Puerta Plata":"Puerto Plata",
    "La Romana": "La Romana",
    "Bayahibe": "Bayahibe",
    "Bayahíbe": "Bayahibe",
    "Cabarete": "Cabarete",
    "Sosua": "Sosua",
    "Sosúa": "Sosua",
    "Samaná": "Samaná",
    "Samana": "Samaná",
    "Bavaro":"Bávaro",
    "Bávaro":"Bávaro",
    "Cap Cana":"Punta Cana",
    "Guayacanes":"Guayacanes",
    "Dominicus":"Dominicus",
    "Gaspar":"Cabarete",
    "La Altagracia": "Bayahibe",
    "Los Melones": "Bayahibe",
    "Sabana":"Samaná",
    "San Pedro de Macoris":"Juan Dolio",
    "San Pedro de Macorís":"Juan Dolio",
    "Santiago":"Santiago",
    "Avenida Charles de Gaulle":"Santo Domingo",
    "Los Ríos":"Santo Domingo"
}

# Definir la función para categorizar la ubicación
def categorizar_ubicacion(row):
    ubicacion = row['UBICACION']
    if ubicacion.lower() in ["no encontrado", "república dominicana"]:
        ubicacion = row['TIPO_ALOJAMIENTO']
    for lugar, ciudad in ubicaciones_a_ciudades.items():
        if lugar.lower() in ubicacion.lower():
            return ciudad
    return row['UBICACION']  # Devolver la misma ubicación si no coincide con ninguna especificada

# Aplicar la función a las filas del DataFrame
df_Hospedajes['UBICACION'] = df_Hospedajes.apply(categorizar_ubicacion, axis=1)


In [15]:
# Verificar el resultado
df_Hospedajes['UBICACION'].unique()

array(['Bayahibe', 'Santo Domingo', 'Puerto Plata', 'La Romana',
       'Guayacanes', 'Juan Dolio', 'Samaná', 'Punta Cana', 'Cabarete',
       'Santiago', 'Boca Chica', 'Bávaro', 'Sosua', 'Dominicus'],
      dtype=object)

Precio

In [17]:
# Definir la función para limpiar el precio
def limpiar_precio(precio):
    if precio == "No encontrado":
        return precio
    else:
        # Extraer solo los caracteres numéricos
        return float(re.sub(r'[^\d.]', '', precio))

# Aplicar la función a la columna 'PRECIO_USD'
df_Hospedajes['PRECIO_USD'] = df_Hospedajes['PRECIO_USD'].apply(limpiar_precio)

In [18]:
# Verificar el resultado
print(df_Hospedajes['PRECIO_USD'].head())

0      48.0
1      65.0
2     600.0
3    2500.0
4      39.0
Name: PRECIO_USD, dtype: object


Calificaciones

In [19]:
# Definir la función para limpiar las calificaciones
def limpiar_calificacion(calificacion):
    try:
        # Intentar convertir a float
        calificacion = float(calificacion)
        return calificacion
    except ValueError:
        # Si no es un número y no es "No encontrado", devolver "No encontrado"
        if calificacion == "No encontrado":
            return calificacion
        else:
            return "No encontrado"

# Aplicar la función a las columnas de calificaciones
calificacion_columnas = ['CALIFICACION_LLEGADA', 'CALIFICACION_COMUNICACION', 'CALIFICACION_UBICACION']
for columna in calificacion_columnas:
    df_Hospedajes[columna] = df_Hospedajes[columna].apply(limpiar_calificacion)

In [20]:
# Verificar el resultado
df_Hospedajes[calificacion_columnas].head(500)

,CALIFICACION_LLEGADA,CALIFICACION_COMUNICACION,CALIFICACION_UBICACION
0,No encontrado,No encontrado,No encontrado
1,4.6,4.8,4.0
2,No encontrado,No encontrado,No encontrado
3,No encontrado,No encontrado,No encontrado
4,4.9,5.0,4.8
...,...,...,...
495,No encontrado,No encontrado,No encontrado
496,5.0,4.9,4.8
497,4.7,4.7,4.8
498,No encontrado,No encontrado,No encontrado


Tipo de alojamiento

In [26]:
# Definir el diccionario de mapeo de tipos de alojamiento
mapeo_tipos_alojamiento = {
    "Apartotel": "Apartotel",
    "Complejo turístico": "Apartotel",
    "Condominio": "Apartotel",
    "Residencia": "Apartotel",
    "Casa de huéspedes": "Hostal",
    "Casa particular": "Hostal",
    "Hostal": "Hostal",
    "Bed and breakfast": "Hostal",
    "Hotel": "Hotel",
    "Hotel boutique": "Hotel",
    "Habitación de hotel": "Hotel",
    "Casa adosada": "Casa",
    "Casa de arcilla": "Casa",
    "Casa de campo": "Casa",
    "Isla": "Casa",
    "Castillo": "Casa",
    "Chalet": "Cabaña",
    "Cabaña": "Cabaña",
    "Bungalow": "Cabaña",
    "Casa rodante": "Cabaña",
    "Minicasa": "Cabaña",
    "Domo": "Cabaña",
    "Suite": "Apartamento",
    "Loft": "Apartamento",
    "Departamento": "Apartamento",
    "Alojamiento": "Alojamiento",
    "Lugar para hospedarte": "Alojamiento",
    "Villa": "Villa",
    "Barco": "Barco",
    "Habitación compartida": "Habitación",
    "Habitación": "Habitación"
}

# Función para ajustar el tipo de alojamiento según la condición especial
def ajustar_tipo_alojamiento(row):
    tipo_alojamiento = row['TIPO_ALOJAMIENTO']
    espacio_alojamiento = row['ESPACIO_ALOJAMIENTO']
    
    # Buscar en el diccionario de mapeo
    for key, value in mapeo_tipos_alojamiento.items():
        if key.lower() in tipo_alojamiento.lower():
            tipo_alojamiento_mapeado = value
            break
    else:
        tipo_alojamiento_mapeado = tipo_alojamiento  # Si no se encuentra, dejar el original

    # Condición especial para habitaciones
    if tipo_alojamiento_mapeado in ["Habitación", "Habitación compartida"] and espacio_alojamiento == "Alojamiento entero":
        return "Casa"
    else:
        return tipo_alojamiento_mapeado

# Aplicar la función a las filas del DataFrame
df_Hospedajes['TIPO_ALOJAMIENTO'] = df_Hospedajes.apply(ajustar_tipo_alojamiento, axis=1)


In [27]:
# Verificar el resultado
df_Hospedajes['TIPO_ALOJAMIENTO'].unique()

array(['Apartamento', 'Habitación', 'Apartotel', 'Casa', 'Hotel', 'Villa',
       'Alojamiento', 'Hostal', 'Cabaña', 'Barco'], dtype=object)

ES_SUPER_ANFITRION

In [29]:
# Reemplazar los valores en la columna 'ES_ANFITRION'
df_Hospedajes['ES_SUPER_ANFITRION'] = df_Hospedajes['ES_SUPER_ANFITRION'].replace({'SI': 1, 'NO': 0})

EXPORT

In [30]:
df_Hospedajes.to_excel('../data_out/Hospedajes_RD-CLEANED.xlsx', index=False)